In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Charger le dataset
data = pd.read_csv("../OLD_SCRIPT/DatasetmalwareExtrait.csv")

# Séparer les caractéristiques (X) et la cible (y)
X = data.drop(columns=["legitimate"])
y = data["legitimate"]

In [3]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Définir le modèle et les hyperparamètres pour la recherche (random forest)
forest_model = RandomForestClassifier(random_state=42)
# param_grid_random_forest = {
#     'n_estimators': [50, 100, 200],  # Nombre d'arbres dans la forêt
#     'max_depth': [10, 20, 30, None], # Profondeur maximale des arbres
#     'min_samples_split': [2, 5, 10], # Nombre minimum d'échantillons pour diviser un nœud
#     'min_samples_leaf': [1, 2, 4]    # Nombre minimum d'échantillons dans une feuille
# }

param_grid_random_forest = {
    'n_estimators': [50, 100],      # Réduction du nombre d'arbres à tester
    'max_depth': [10, 20],          # Limitation à deux options pour la profondeur
    'min_samples_split': [2, 5],    # Réduction des options pour les divisions
    'min_samples_leaf': [1, 2]      # Réduction des tailles minimales de feuilles
}


# Définir le modèle et les hyperparamètres pour la recherche (decision tree)
tree_model = DecisionTreeClassifier(random_state=42)
param_grid_decision_tree = {
    'criterion': ['gini', 'entropy'],        # Fonction pour mesurer la qualité du split
    'max_depth': [10, 20, 30, None],         # Profondeur maximale de l'arbre
    'min_samples_split': [2, 5, 10],         # Nombre minimum d'échantillons pour diviser un nœud
    'min_samples_leaf': [1, 2, 4],           # Nombre minimum d'échantillons dans une feuille
    'max_features': [None, 'sqrt', 'log2']   # Nombre maximum de caractéristiques pour chercher un split
}


In [5]:
# Effectuer la recherche avec validation croisée (random forest)
grid_search_random_forest = GridSearchCV(estimator=forest_model, param_grid=param_grid_random_forest, cv=5, scoring='accuracy')
grid_search_random_forest.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [10, 20], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 100]},
             scoring='accuracy')

In [6]:
# Effectuer la recherche avec validation croisée (decision tree)
grid_search_decision_tree = GridSearchCV(estimator=tree_model, param_grid=param_grid_decision_tree, cv=5, scoring='accuracy')
grid_search_decision_tree.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, 30, None],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [7]:
# Meilleurs hyperparamètres trouvés (random forest)
print("Meilleurs hyperparamètres :", grid_search_random_forest.best_params_)

Meilleurs hyperparamètres : {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [8]:
# Meilleurs hyperparamètres trouvés (decision tree)
print("Meilleurs hyperparamètres :", grid_search_decision_tree.best_params_)

Meilleurs hyperparamètres : {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}


In [9]:
# Prédictions sur l'ensemble de test (random forest)
y_pred_random_forest = grid_search_random_forest.best_estimator_.predict(X_test)

In [10]:
# Prédictions sur l'ensemble de test (decision tree)
y_pred_decision_tree = grid_search_decision_tree.best_estimator_.predict(X_test)

In [11]:
# Rapport de classification random forest
print("\nRapport de classification :\n")
print(classification_report(y_test, y_pred_random_forest))


Rapport de classification :

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     19186
           1       0.99      0.99      0.99      8303

    accuracy                           0.99     27489
   macro avg       0.99      0.99      0.99     27489
weighted avg       0.99      0.99      0.99     27489



In [17]:
# Rapport de classification decision tree
print("\nRapport de classification :\n")
print(classification_report(y_test, y_pred_decision_tree))


Rapport de classification :

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     19186
           1       0.98      0.98      0.98      8303

    accuracy                           0.99     27489
   macro avg       0.99      0.99      0.99     27489
weighted avg       0.99      0.99      0.99     27489



In [13]:
# Méthode pour évaluer le modèle
def evaluate_model(model, X_test, y_test):
    accuracy = accuracy_score(y_test, model.predict(X_test))
    print(f"\nPrécision du modèle : {accuracy:.2f}")
    return accuracy

In [14]:
# Évaluer le modèle random_forest avec grid_search
evaluate_model(grid_search_random_forest.best_estimator_, X_test, y_test)


Précision du modèle : 0.99


0.991742151406017

In [15]:
# Évaluer le modèle decision tree avec grid_search
evaluate_model(grid_search_decision_tree.best_estimator_, X_test, y_test)


Précision du modèle : 0.99


0.9885044927061734